In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
# pd.set_option('display.max_columns', None) # 전체 열 보기
# pd.set_option('display.max_rows', None) # 전체 행 보기

In [3]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [4]:
train_data.describe()

,Insp. Seq No._Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Unit Time_Dam,CURE END POSITION Z Judge Value_Dam,CURE END POSITION ? Collect Result_Dam,CURE END POSITION ? Unit Time_Dam,CURE END POSITION ? Judge Value_Dam,...,PalletID Judge Value_Fill2,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2
count,40506.0,40506.000000,0.0,0.0,40506.000000,0.0,0.0,40506.000000,0.0,0.0,...,0.0,40506.000000,0.0,0.0,40506.000000,0.0,0.0,40506.000000,0.0,0.0
mean,1.0,530.370809,NaN,NaN,6.320669,NaN,NaN,-21.227966,NaN,NaN,...,NaN,73.539648,NaN,NaN,108.691453,NaN,NaN,0.771392,NaN,NaN
std,0.0,369.283055,NaN,NaN,4.858988,NaN,NaN,87.461776,NaN,NaN,...,NaN,105.176311,NaN,NaN,130.758018,NaN,NaN,1.289203,NaN,NaN
min,1.0,240.000000,NaN,NaN,2.500000,NaN,NaN,-90.000000,NaN,NaN,...,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN
25%,1.0,240.000000,NaN,NaN,2.500000,NaN,NaN,-90.000000,NaN,NaN,...,NaN,7.000000,NaN,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN
50%,1.0,240.000000,NaN,NaN,2.500000,NaN,NaN,-90.000000,NaN,NaN,...,NaN,13.000000,NaN,NaN,49.000000,NaN,NaN,1.000000,NaN,NaN
75%,1.0,1000.000000,NaN,NaN,12.500000,NaN,NaN,90.000000,NaN,NaN,...,NaN,120.000000,NaN,NaN,196.000000,NaN,NaN,1.000000,NaN,NaN
max,1.0,1000.000000,NaN,NaN,12.500000,NaN,NaN,90.000000,NaN,NaN,...,NaN,487.000000,NaN,NaN,608.000000,NaN,NaN,17.000000,NaN,NaN


In [5]:
sum = 0 
for i in train_data.columns:
    print(i, " : " , train_data[i].nunique())
    if train_data[i].nunique() > 1 :
        sum += 1

Wip Line_Dam  :  1
Process Desc._Dam  :  1
Equipment_Dam  :  2
Model.Suffix_Dam  :  7
Workorder_Dam  :  663
Insp. Seq No._Dam  :  1
Insp Judge Code_Dam  :  1
CURE END POSITION X Collect Result_Dam  :  2
CURE END POSITION X Unit Time_Dam  :  0
CURE END POSITION X Judge Value_Dam  :  0
CURE END POSITION Z Collect Result_Dam  :  2
CURE END POSITION Z Unit Time_Dam  :  0
CURE END POSITION Z Judge Value_Dam  :  0
CURE END POSITION ? Collect Result_Dam  :  2
CURE END POSITION ? Unit Time_Dam  :  0
CURE END POSITION ? Judge Value_Dam  :  0
CURE SPEED Collect Result_Dam  :  5
CURE SPEED Unit Time_Dam  :  0
CURE SPEED Judge Value_Dam  :  0
CURE STANDBY POSITION X Collect Result_Dam  :  1
CURE STANDBY POSITION X Unit Time_Dam  :  0
CURE STANDBY POSITION X Judge Value_Dam  :  0
CURE STANDBY POSITION Z Collect Result_Dam  :  1
CURE STANDBY POSITION Z Unit Time_Dam  :  0
CURE STANDBY POSITION Z Judge Value_Dam  :  0
CURE STANDBY POSITION ? Collect Result_Dam  :  1
CURE STANDBY POSITION ? Unit Time_

THICKNESS 2 Collect Result_Dam  :  7
THICKNESS 2 Unit Time_Dam  :  0
THICKNESS 2 Judge Value_Dam  :  0
THICKNESS 3 Collect Result_Dam  :  8
THICKNESS 3 Unit Time_Dam  :  0
THICKNESS 3 Judge Value_Dam  :  0
WorkMode Collect Result_Dam  :  8
WorkMode Unit Time_Dam  :  0
WorkMode Judge Value_Dam  :  0
Wip Line_AutoClave  :  1
Process Desc._AutoClave  :  1
Equipment_AutoClave  :  1
Model.Suffix_AutoClave  :  7
Workorder_AutoClave  :  663
Insp. Seq No._AutoClave  :  1
Insp Judge Code_AutoClave  :  1
1st Pressure Collect Result_AutoClave  :  27
1st Pressure 1st Pressure Unit Time_AutoClave  :  9
1st Pressure Judge Value_AutoClave  :  1
2nd Pressure Collect Result_AutoClave  :  67
2nd Pressure Unit Time_AutoClave  :  10
2nd Pressure Judge Value_AutoClave  :  1
3rd Pressure Collect Result_AutoClave  :  67
3rd Pressure Unit Time_AutoClave  :  11
3rd Pressure Judge Value_AutoClave  :  1
Chamber Temp. Collect Result_AutoClave  :  26
Chamber Temp. Unit Time_AutoClave  :  24
Chamber Temp. Judge Val

In [6]:
print(sum)

146


In [7]:
for column in train_data.columns:
    if train_data[column].nunique() == 1 or train_data[column].nunique() == 0:
        train_data.drop(column, axis=1, inplace=True)

In [8]:
for i in train_data.columns:
    print(i, " : " , train_data[i].nunique())

Equipment_Dam  :  2
Model.Suffix_Dam  :  7
Workorder_Dam  :  663
CURE END POSITION X Collect Result_Dam  :  2
CURE END POSITION Z Collect Result_Dam  :  2
CURE END POSITION ? Collect Result_Dam  :  2
CURE SPEED Collect Result_Dam  :  5
CURE START POSITION X Collect Result_Dam  :  2
CURE START POSITION ? Collect Result_Dam  :  2
DISCHARGED SPEED OF RESIN Collect Result_Dam  :  3
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam  :  19
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam  :  29
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam  :  20
Dispense Volume(Stage1) Collect Result_Dam  :  23
Dispense Volume(Stage2) Collect Result_Dam  :  33
Dispense Volume(Stage3) Collect Result_Dam  :  22
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam  :  7
HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam  :  41
HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam  :  40
HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam  :  36
HEAD NORMAL COORDINATE Y AXIS(Stage2)

In [17]:
for i in train_data.columns:
    print(i, " : " , train_data[i].unique())

Equipment_Dam  :  ['Dam dispenser #1' 'Dam dispenser #2']
Model.Suffix_Dam  :  ['AJX75334505' 'AJX75334501' 'AJX75334502' 'AJX75334507' 'AJX75334506'
 'AJX75334508' 'AJX75334503']
Workorder_Dam  :  ['4F1XA938-1' '3KPM0016-2' '4E1X9167-1' '3K1X0057-1' '3HPM0007-1'
 '3G1X8297-2' '3M1XH425-1' '4A1XB542-4' '3H1X9883-1' '3G1XD171-2'
 '4E1X9168-1' '4A1XC343-1' '3L1XA128-1' '3L1X8574-2' '3GPM0070-1'
 '3G1X4502-2' '3F1X9648-1' '4B1XA868-1' '3L1XB019-1' '4C1X0049-1'
 '4B1XD472-2' '4B1XD691-1' '3G1XB994-1' '4B1XA871-1' '3MPXX030-0001'
 '3H1XD883-1' '3M1XC472-1' '3H1X9881-1' '3I1XB425-1' '4B1X9674-1'
 '3G1XC498-1' '4A1XB975-2' '3M1XF736-1' '4B1XC717-1' '4C1XG294-1'
 '3G1XC759-1' '3J1XF461-1' '3L1XA998-1' '3K1X0791-1' '3F1XB560-1'
 '3J1XC148-1' '4E1X9997-2' '3I1XB274-1' '3K1XA736-1' '4C1XH941-1'
 '4C1XH315-1' '4A1XC346-1' '3L1X9934-1' '4F1X8043-1' '3H1XE851-1'
 '4F1XB043-1' '3I1XA510-1' '3K1X9801-1' '3L1XB735-2' '3M1XH386-1'
 '3G1XC632-1' '4C1XH944-2' '4B1XC718-2' '3J1XD560-1' '4B1X9672-1'
 '4E1X9

In [18]:
for i in train_data.columns:
    if(train_data[i].nunique() <= 20):
        print(i, " : " , train_data[i].unique())


Equipment_Dam  :  ['Dam dispenser #1' 'Dam dispenser #2']
Model.Suffix_Dam  :  ['AJX75334505' 'AJX75334501' 'AJX75334502' 'AJX75334507' 'AJX75334506'
 'AJX75334508' 'AJX75334503']
CURE END POSITION X Collect Result_Dam  :  [ 240 1000]
CURE END POSITION Z Collect Result_Dam  :  [ 2.5 12.5]
CURE END POSITION ? Collect Result_Dam  :  [-90  90]
CURE SPEED Collect Result_Dam  :  [100  70  85  95 105]
CURE START POSITION X Collect Result_Dam  :  [1030  280]
CURE START POSITION ? Collect Result_Dam  :  [-90  90]
DISCHARGED SPEED OF RESIN Collect Result_Dam  :  [16 10 15]
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam  :  [14.9 21.3 14.7  9.7  9.6 13.2 14.2 13.6 17.  13.5 13.1 21.2 13.8 11.6
 14.8 14.3 17.1 13.7 10.6]
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam  :  [14.7 21.3  9.6  9.7 13.2 14.3 13.1 13.6 14.2 17.  17.1 13.5 13.8 21.2
 11.6 14.8 13.7  9.8 21.4 11.5]
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam  :  [nan '550.3' 'OK' '162.4' '549' '549.5' '550' '548.5']


In [10]:
train_data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 146 columns):
 #    Column                                                      Non-Null Count  Dtype  
---   ------                                                      --------------  -----  
 0    Equipment_Dam                                               40506 non-null  object 
 1    Model.Suffix_Dam                                            40506 non-null  object 
 2    Workorder_Dam                                               40506 non-null  object 
 3    CURE END POSITION X Collect Result_Dam                      40506 non-null  int64  
 4    CURE END POSITION Z Collect Result_Dam                      40506 non-null  float64
 5    CURE END POSITION ? Collect Result_Dam                      40506 non-null  int64  
 6    CURE SPEED Collect Result_Dam                               40506 non-null  int64  
 7    CURE START POSITION X Collect Result_Dam                    40506 non-null

C:\Users\kushn\AppData\Local\Temp\ipykernel_23180\3200127547.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train_data.info(verbose = True, null_counts = True)


In [11]:
train_data.dtypes[train_data.dtypes=='object']

Equipment_Dam                                                 object
Model.Suffix_Dam                                              object
Workorder_Dam                                                 object
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      object
Model.Suffix_AutoClave                                        object
Workorder_AutoClave                                           object
Chamber Temp. Judge Value_AutoClave                           object
Equipment_Fill1                                               object
Model.Suffix_Fill1                                            object
Workorder_Fill1                                               object
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    object
Equipment_Fill2                                               object
Model.Suffix_Fill2                                            object
Workorder_Fill2                                               object
HEAD NORMAL COORDINATE X AXIS(Stag

In [12]:
train_data.select_dtypes(include = 'object')

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,Model.Suffix_AutoClave,Workorder_AutoClave,Chamber Temp. Judge Value_AutoClave,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,NaN,AJX75334505,4F1XA938-1,OK,Fill1 dispenser #1,AJX75334505,4F1XA938-1,NaN,Fill2 dispenser #1,AJX75334505,4F1XA938-1,NaN,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,550.3,AJX75334505,3KPM0016-2,OK,Fill1 dispenser #1,AJX75334505,3KPM0016-2,838.4,Fill2 dispenser #1,AJX75334505,3KPM0016-2,835.5,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,OK,AJX75334501,4E1X9167-1,OK,Fill1 dispenser #2,AJX75334501,4E1X9167-1,OK,Fill2 dispenser #2,AJX75334501,4E1X9167-1,OK,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,162.4,AJX75334501,3K1X0057-1,OK,Fill1 dispenser #2,AJX75334501,3K1X0057-1,837.7,Fill2 dispenser #2,AJX75334501,3K1X0057-1,305,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,549,AJX75334501,3HPM0007-1,NG,Fill1 dispenser #1,AJX75334501,3HPM0007-1,838.4,Fill2 dispenser #1,AJX75334501,3HPM0007-1,835.5,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,550.3,AJX75334501,3J1XF434-2,OK,Fill1 dispenser #1,AJX75334501,3J1XF434-2,838.4,Fill2 dispenser #1,AJX75334501,3J1XF434-2,835.5,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,NaN,AJX75334501,4E1XC796-1,NG,Fill1 dispenser #2,AJX75334501,4E1XC796-1,NaN,Fill2 dispenser #2,AJX75334501,4E1XC796-1,NaN,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,OK,AJX75334501,4C1XD438-1,NG,Fill1 dispenser #1,AJX75334501,4C1XD438-1,OK,Fill2 dispenser #1,AJX75334501,4C1XD438-1,OK,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,162.4,AJX75334501,3I1XA258-1,OK,Fill1 dispenser #2,AJX75334501,3I1XA258-1,837.7,Fill2 dispenser #2,AJX75334501,3I1XA258-1,305,Normal


In [13]:
train_data.select_dtypes(exclude = 'object')

,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION ? Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION ? Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,...,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,240,2.5,-90,100,1030,-90,16,14.9,8.4,14.7,...,119,50.0,91.8,270,50,114.612,19.9,7,127,1
1,240,2.5,-90,70,1030,-90,10,21.3,4.9,21.3,...,50,91.8,270.0,50,85,19.600,7.0,185,1,0
2,1000,12.5,90,85,280,90,16,14.7,8.5,14.7,...,119,50.0,91.8,270,50,114.612,19.8,10,73,1
3,1000,12.5,90,70,280,90,10,21.3,8.4,21.3,...,50,91.8,270.0,50,85,19.900,12.0,268,1,0
4,240,2.5,-90,70,1030,-90,10,9.7,4.9,9.6,...,50,91.8,270.0,50,85,19.700,8.0,121,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,240,2.5,-90,70,1030,-90,10,17.0,5.0,17.0,...,50,91.8,270.0,50,85,19.200,1.0,318,1,0
40502,1000,12.5,90,100,280,90,16,14.9,8.5,14.7,...,119,50.0,91.8,270,50,114.612,20.5,14,197,1
40503,240,2.5,-90,100,1030,-90,16,14.2,8.2,14.3,...,119,50.0,91.8,270,50,85.000,19.7,1,27,1
40504,1000,12.5,90,70,280,90,10,9.7,4.9,9.7,...,50,91.8,270.0,50,85,20.100,13.0,117,1,0
